# Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import polars as pl
import pandas as pd
import numpy as np
import json
import os
import scipy.stats as stats
import seaborn as sns

In [ ]:
with open('../params.json', 'r') as file :
    params = json.load(file)

DATASET, VERSION = params['dataset'], params['version']
DATA_FOLD = params['data_folder']

In [ ]:
DIR = f'{DATA_FOLD}/{VERSION}/3.analysis/imputation_48/{DATASET}/tables/'
print({DATASET})

# Datasets

In [ ]:
df_mean = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_mean.xlsx')
df_lin = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_lin_interpol.xlsx')
df_saits = pd.read_excel(DIR + '/pam_comparaison/' + 'pa_only_saits.xlsx')

In [ ]:
df_lin.head()

In [ ]:
df_mean.shape == df_lin.shape == df_saits.shape

#  Bland - Altman function

In [ ]:
sns.set_theme(style="whitegrid", context="paper", font_scale=1.2)

mpl.rcParams.update({
    "font.family"      : "serif",
    "font.serif"       : ["Times New Roman"],
    "axes.titlesize"   : "medium",
    "axes.labelsize"   : "medium",
    "xtick.labelsize"  : "small",
    "ytick.labelsize"  : "small",
    "figure.dpi"       : 300,
    "axes.spines.right": False,
    "axes.spines.top"  : False,
})

In [ ]:
sns.set_style('whitegrid')

fig, axes = plt.subplots(3, figsize = (16,12))

datasets = [
    (df_mean, "A : Mean", axes[0]),
    (df_lin, "B : Linear Interpolation", axes[1]),
    (df_saits, "C : SAITS", axes[2]),
]

for df, title, ax in datasets:
    x = 'masquées'
    y= 'différence'
    values = np.vstack([df[x], df[y]])
    kernel = stats.gaussian_kde(values)(values)
    sns.scatterplot(data = df, x=x, y=y, ax=ax, c=kernel, cmap='viridis')
    #twin_ax= ax.twinx()
    #sns.kdeplot(data=df, x=x, ax=twin_ax, color='green')
    #sns.scatterplot(data = df, x='masquées', y='différence', ax=ax, alpha=0.5)

    sc = ax.scatter(
        df[x], df[y],
        c=kernel, cmap='viridis',
        s=20, edgecolors='none'
)

    # add a vertical colour bar on the **right** of THIS subplot
    cbar = fig.colorbar(sc, ax=ax, pad=0.01, orientation='vertical')
    ax.set_ylim(-80, 80)
    ax.set_xlim(20, 140)
    
    #twin_ax.set_ylim(0,0.2)
    ax.set_xlabel('Average of Imputed and Msked Values (mmHg)')
    ax.set_ylabel('Imputed Value - Masked Values (mmHg)')
    ax.set_title(title, loc='left')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib import colors
from matplotlib.lines import Line2D

sns.set_style("whitegrid")
cmap = sns.color_palette("crest", as_cmap=True)
norm = colors.Normalize(vmin=0, vmax=1)
fig, axes = plt.subplots(3, figsize=(8.27, 11.69), sharex=True, sharey=True)

datasets = [
    (df_mean,  "A : Mean"),
    (df_lin,   "B : Linear Interpolation"),
    (df_saits, "C : SAITS"),
]

for ax, (df, title) in zip(axes, datasets):
    x, y = "moyenne", "différence"
    values = np.vstack([df[x], df[y]])

    # KDE scaled so that max density = 1
    kernel = stats.gaussian_kde(values)(values)
    kernel /= kernel.max()

    sc = ax.scatter(
        df[x], df[y],
        c=kernel, cmap=cmap, norm=norm,
        s=20, edgecolors="none"
    )

    # compact colour bar without ticks
    cbar = fig.colorbar(sc, ax=ax, pad=0.01, orientation="vertical")
    cbar.set_ticks([])
    cbar.ax.tick_params(size=0)
    cbar.set_label("Relative density")

    mean_diff = df[y].mean()
    sd_diff   = df[y].std(ddof=1)
    upper     = mean_diff + 1.96 * sd_diff
    lower     = mean_diff - 1.96 * sd_diff

    ax.axhline(mean_diff, color="red",   linewidth=1.5)           # mean
    ax.axhline(upper,     color="black", linewidth=1.2, ls="--")  # +1.96 SD
    ax.axhline(lower,     color="black", linewidth=1.2, ls="--")  # –1.96 SD
    handles = [
        Line2D([0], [0], color="red",   lw=1.5),
        Line2D([0], [0], color="black", lw=1.2, ls="--"),
        Line2D([0], [0], color="black", lw=1.2, ls="--"),
    ]
    labels = [
        f"Mean = {mean_diff: .2f} mmHg",
        f"+1.96 SD = {upper: .2f} mmHg",
        f"−1.96 SD = {lower: .2f} mmHg",
    ]
    ax.legend(
        handles, labels,
        loc="upper right",             # inside the axes
        frameon=True, framealpha=0.9,
        fontsize="small",
        borderpad=0.4
    )

    # limits and labels
    ax.set_xlim(20, 140)
    ax.set_ylim(-80, 80)
    ax.set_ylabel("Imputed – Masked Values (mmHg)")

    # keep x-label only on the bottom panel
    if ax is axes[-1]:
        ax.set_xlabel("Average of Imputed and Masked Values (mmHg)")
    else:
        ax.set_xlabel("")
        ax.tick_params(axis="x", labelbottom=False)

    # internal title, top-left corner of the axes
    ax.text(
        0.02, 0.98, title,
        transform=ax.transAxes,
        ha="left", va="top",
        fontsize="medium", fontweight="bold"
    )

fig.tight_layout()


In [ ]:
fig.savefig()